In [2]:
#for installing conda on google colab

!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh 
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh  -b -f -p /usr/local

import sys
sys.path.append('usr/local/python3.7/site-packages/')

--2020-06-17 08:06:39--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   268MB/s    in 0.3s    

2020-06-17 08:06:40 (268 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [3]:
# clone torchreid repo
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create conda environment
import os
os.chdir('deep-person-reid')
!conda create --name torchreid python=3.7
!conda activate torchreid

# install dependencies
!pip install -r requirements.txt

# install torch and torchvision (select the proper cuda version to suit your machine)
!conda install pytorch torchvision cudatoolkit=9.0 -c pytorch

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 8059, done.
remote: Total 8059 (delta 0), reused 0 (delta 0), pack-reused 8059
Receiving objects: 100% (8059/8059), 8.55 MiB | 10.85 MiB/s, done.
Resolving deltas: 100% (5893/5893), done.
Solving environment: \ done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local/envs/torchreid

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.2         |           py37_0         157 KB
    libffi-3.3                 |       he6710b0_1          50 KB
    ncurses-6.2                |       he6710b0_1         817 KB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    pip-20.1.1   

Solving environment: / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=9.0
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    conda-4.8.3                |           py37_0         2.8 MB
    cudatoolkit-9.0            |       h13b8566_0       237.0 MB
    freetype-2.9.1             |       h8a8886c_1         550 KB
    intel-openmp-2020.1        |              217         780 KB
    jpeg-9b                    |       h024ee3a_2         214 KB
    libgfortran-ng-7.3.0       |       hdf63c60_0        1006 KB
    libpng-1.6.37              |       hbc83047_0         278 KB
    libtiff-4.1.0              |       h2733197_0         447 KB
    mkl-2020.1                 |              217       129.0 MB
    mkl-servic

**Make sure runtine is changed to GPU**

# **Person Re-ID Model Training & Testing**


In [4]:
import torchreid


/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '


In [5]:
torchreid.models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0']


Loading the data manager.


In [6]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources=['market1501'],
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=32,
    transforms = ['random_flip', 'random_crop']
)



Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 8713 KB/s, 17 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

Building model 

In [ ]:
model = torchreid.models.build_model(
    name = 'osnet_x1_0',
    num_classes = datamanager.num_train_pids,
    loss = 'softmax',
    pretrained = True,
    use_gpu = True
)

model = model.cuda()

Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
10.9MB [00:00, 295MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003,
    weight_decay = 0.0005,
    momentum=0.9,
    sgd_dampening = 0,
    rmsprop_alpha = 0.99,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler= 'single_step',
    stepsize=20
)



In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer = optimizer,
    scheduler = scheduler,
    label_smooth = True
)


In [ ]:
engine.run(
    save_dir = 'log/osnet_market_1501',
    max_epoch = 10,
    eval_freq = 10,
    print_freq = 10,
)


=> Start training
epoch: [1/10][10/404]	time 0.173 (0.270)	data 0.000 (0.060)	eta 0:18:06	loss 1.1281 (1.1965)	acc 100.0000 (99.0625)	lr 0.000300
epoch: [1/10][20/404]	time 0.178 (0.222)	data 0.000 (0.030)	eta 0:14:52	loss 1.2265 (1.1939)	acc 100.0000 (99.3750)	lr 0.000300
epoch: [1/10][30/404]	time 0.177 (0.208)	data 0.000 (0.020)	eta 0:13:52	loss 1.1445 (1.1864)	acc 100.0000 (99.4792)	lr 0.000300
epoch: [1/10][40/404]	time 0.175 (0.199)	data 0.000 (0.015)	eta 0:13:14	loss 1.2932 (1.1872)	acc 93.7500 (99.2188)	lr 0.000300
epoch: [1/10][50/404]	time 0.184 (0.194)	data 0.000 (0.012)	eta 0:12:54	loss 1.2598 (1.1923)	acc 96.8750 (99.1250)	lr 0.000300
epoch: [1/10][60/404]	time 0.173 (0.191)	data 0.000 (0.010)	eta 0:12:38	loss 1.3095 (1.1937)	acc 96.8750 (99.1667)	lr 0.000300
epoch: [1/10][70/404]	time 0.172 (0.189)	data 0.000 (0.009)	eta 0:12:29	loss 1.1616 (1.1981)	acc 100.0000 (99.1518)	lr 0.000300
epoch: [1/10][80/404]	time 0.173 (0.187)	data 0.000 (0.008)	eta 0:12:19	loss 1.2051 (1.19

In [ ]:
#Resume training for improving mAP and CMC

start_epoch = torchreid.utils.resume_from_checkpoint(
    'log/osnet_market_1501/model/model.pth.tar-10',
    model,
    optimizer
)

engine.run(
    save_dir = 'log/osnet_market_1501',
    max_epoch = 30,
    start_epoch=start_epoch
)

Loading checkpoint from "log/osnet_market_1501/model/model.pth.tar-10"
Loaded model weights
Loaded optimizer
Last epoch = 10
Last rank1 = 88.9%
=> Start training
epoch: [11/30][10/404]	time 0.166 (0.250)	data 0.000 (0.062)	eta 0:33:38	loss 1.0739 (1.0933)	acc 100.0000 (100.0000)	lr 0.000300
epoch: [11/30][20/404]	time 0.161 (0.206)	data 0.000 (0.031)	eta 0:27:43	loss 1.0950 (1.0990)	acc 100.0000 (99.8438)	lr 0.000300
epoch: [11/30][30/404]	time 0.158 (0.193)	data 0.000 (0.021)	eta 0:25:50	loss 1.1025 (1.0965)	acc 100.0000 (99.7917)	lr 0.000300
epoch: [11/30][40/404]	time 0.169 (0.185)	data 0.000 (0.016)	eta 0:24:50	loss 1.0919 (1.0988)	acc 100.0000 (99.6875)	lr 0.000300
epoch: [11/30][50/404]	time 0.162 (0.181)	data 0.000 (0.013)	eta 0:24:11	loss 1.0727 (1.0986)	acc 100.0000 (99.7500)	lr 0.000300
epoch: [11/30][60/404]	time 0.182 (0.178)	data 0.000 (0.010)	eta 0:23:48	loss 1.0869 (1.0996)	acc 100.0000 (99.7396)	lr 0.000300
epoch: [11/30][70/404]	time 0.160 (0.176)	data 0.000 (0.009)	et

In [ ]:
#Resume training for improving mAP and CMC
start_epoch = torchreid.utils.resume_from_checkpoint(
    'log/osnet_market_1501/model/model.pth.tar-30',
    model,
    optimizer
)

engine.run(
    save_dir = 'log/osnet_market_1501',
    max_epoch = 60,
    start_epoch=start_epoch
)

Loading checkpoint from "log/osnet_market_1501/model/model.pth.tar-30"
Loaded model weights
Loaded optimizer
Last epoch = 30
Last rank1 = 90.8%
=> Start training
epoch: [31/60][10/404]	time 0.162 (0.248)	data 0.000 (0.057)	eta 0:49:59	loss 1.0309 (1.0348)	acc 100.0000 (100.0000)	lr 0.000030
epoch: [31/60][20/404]	time 0.170 (0.207)	data 0.000 (0.028)	eta 0:41:45	loss 1.0330 (1.0330)	acc 100.0000 (100.0000)	lr 0.000030
epoch: [31/60][30/404]	time 0.168 (0.193)	data 0.000 (0.019)	eta 0:38:51	loss 1.0243 (1.0313)	acc 100.0000 (100.0000)	lr 0.000030
epoch: [31/60][40/404]	time 0.162 (0.186)	data 0.000 (0.014)	eta 0:37:22	loss 1.0317 (1.0331)	acc 100.0000 (100.0000)	lr 0.000030
epoch: [31/60][50/404]	time 0.170 (0.181)	data 0.000 (0.011)	eta 0:36:28	loss 1.0265 (1.0323)	acc 100.0000 (100.0000)	lr 0.000030
epoch: [31/60][60/404]	time 0.160 (0.179)	data 0.000 (0.010)	eta 0:35:56	loss 1.0430 (1.0325)	acc 100.0000 (100.0000)	lr 0.000030
epoch: [31/60][70/404]	time 0.160 (0.177)	data 0.000 (0.00

In [ ]:
!tensorboard --logdir= log/osnet_x1_0_market1501_softmax_cosinelr